# Analyze nucleus for throughput calculation

Data: 
- Spheroid: id551_MDCK_unit_on_on_01_300Hz_405_250mW_488_150mW_488_X2_100-400.npy
- Nucleus: id551_MDCK_unit_on_on_01_300Hz_405_250mW_488_150mW_405_X2_100-400.npy

## Libraries

In [ ]:
import analysis
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from skimage.color import label2rgb
from skimage.transform import rescale

## Parameters

In [ ]:
path = '../../result/'
suffix= ['_X2']
description = '_100-400'
scale = np.repeat(0.295*4, 3) # spatial resolution in μm scale

## Analysis

### Spheroid

In [ ]:
data = 'id551_MDCK_unit_on_on_01_300Hz_405_250mW_488_150mW_488'

In [ ]:
segmented_spheroid = analysis.segment(data=path+data, suffix=suffix, description=description, 
                                      sigma_spot_detection = 7, sigma_outline = 2, rescaling_factor=0.25)

In [ ]:
np.unique(segmented_spheroid[0])

In [ ]:
props_spheroid = analysis.calculate_props(segmented=segmented_spheroid, suffix=suffix, target=('spheroid'), scale=scale)

In [ ]:
props_spheroid[0]

In [ ]:
segmented_spheroid_filtered = analysis.filter_spheroid(segmented=segmented_spheroid, props=props_spheroid, volume_thresh=30000, solidity_thresh=0.7)

In [ ]:
print('The number of spheroid is:', len(np.unique(segmented_spheroid_filtered[0]))-1)

In [ ]:
img_488 = np.load(path+data+suffix[0]+description+'.npy')
img_488 = rescale(img_488, 0.25, anti_aliasing=True)

In [ ]:
labels = label2rgb(segmented_spheroid_filtered[0][50, ...])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img_488[50, ...])
ax[1].imshow(labels)

In [ ]:
labels = label2rgb(segmented_spheroid[0][50, ...])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img_488[50, ...])
ax[1].imshow(labels)

In [ ]:
np.save(path+data+suffix[0]+description+'_segmented.npy', segmented_spheroid[0])
np.save(path+data+suffix[0]+description+'_segmented_filtered.npy', segmented_spheroid_filtered[0])
props_spheroid[0].to_csv(path+data+suffix[0]+description+'_props.csv')

### Nucleus

In [ ]:
data = 'id551_MDCK_unit_on_on_01_300Hz_405_250mW_488_150mW_405'

In [ ]:
segmented_nucleus = analysis.segment(data=path+data, suffix=suffix, description=description, sigma_spot_detection = 1.5, sigma_outline = 0, rescaling_factor=0.25)

In [ ]:
print(segmented_spheroid_filtered[0].shape)
print(segmented_nucleus[0].shape)

In [ ]:
segmented_nucleus_filtered = analysis.filter_nucleus(nucleus=segmented_nucleus, spheroid=segmented_spheroid_filtered, suffix=suffix)

In [ ]:
img_405 = np.load(path+data+suffix[0]+description+'.npy')
img_405 = rescale(img_405, 0.25, anti_aliasing=True)

In [ ]:
labels_spheroid = label2rgb(segmented_spheroid_filtered[0][50, :200, :200])
labels_nucleus = label2rgb(segmented_nucleus[0][50, :200, :200])
labels_nucleus_filtered = label2rgb(segmented_nucleus_filtered[0][50, :200, :200])

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0, 0].imshow(img_405[50, :200, :200])
ax[0, 1].imshow(labels_spheroid)
ax[1, 0].imshow(labels_nucleus)
ax[1, 1].imshow(labels_nucleus_filtered)

In [ ]:
props_nucleus = analysis.calculate_props(segmented=segmented_nucleus_filtered, suffix=suffix, target=('nucleus'), scale=scale)

In [ ]:
props_nucleus

In [ ]:
np.save(path+data+suffix[0]+description+'_segmented.npy', segmented_nucleus[0])
np.save(path+data+suffix[0]+description+'_segmented_filtered.npy', segmented_nucleus[0])
props_nucleus.to_csv(path+data+suffix[0]+description+'_props.csv')

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

columns = ['Volume', 'Surface area', 'Surface-area-to-Volume', 'Aspect ratio', 'Solidity']

for i, ax in enumerate(axes):
    if i == 5:
        axes[i].set_axis_off()
    else:
        sns.violinplot(data=props_nucleus, y=columns[i], ax=ax)

plt.subplots_adjust(wspace=0.35, hspace=0.3)